I used a ipynb file here to make it easier to run section-by-section and see how it proceeds after each step. You can just combine all of them in a .py file.

Feel free to change any coding or the main regression model. Obviously I'm here for any questions you might have.

I think it's easy to adapt this to other stock markets. I'm going to add the notepad file of each market's open and close, and location. You can edit different variables here to adapt it to other markets. Beware of coordinates, and timezone.

In [3]:
#Preparing and shaping the data
import pandas as pd
import numpy as np
from geopy.distance import geodesic

df_eq = pd.read_csv('data/test/EQtest.csv') #EQtest is the data set including earthquakes equal or larger than 6 from August 1, 2008 to the end of 2009
df_n225 = pd.read_csv('data/test/^N225.csv') #^N225.csv is the daily changes in N225 stock market from August 1, 2008 to the end of 2009 (the first two files)
df_n225.rename(columns={'Date': 'date'}, inplace=True) #The original file has it as Date, but changing to date makes it much easier

# Convert df_n225 date column to Japan Standard Time Zone (JST)
df_n225['date'] = pd.to_datetime(df_n225['date']).dt.tz_localize('Japan')

# Convert df_eq date column and normalize times to 15:25 JST
df_eq['date'] = pd.to_datetime(df_eq['date'], format='mixed')
df_eq['date_1600'] = df_eq['date'].dt.normalize() + pd.Timedelta(hours=15, minutes=25) #Setting the end of trade on N225 market

# Define Tokyo coordinates
tokyo_coords = (36.6528, 139.8394)

# Initialize lists for new columns
num_list, sum_list, max_list, avg_list, min_dist_list, max_dist_list = [], [], [], [], [], []

# Process each row in df_n225
for i, n225_row in df_n225.iterrows():
    curr_date = n225_row['date']
    prev_date = df_n225.iloc[i - 1]['date'] if i > 0 else None

    # Define time window: after previous day's 15:25 and before current day's 15:25, to make a summary of earthquakes between consecutive end of trade points
    if prev_date is not None:
        eq_filtered = df_eq[(df_eq['date'] > prev_date + pd.Timedelta(hours=15, minutes=25)) & 
                             (df_eq['date'] <= curr_date + pd.Timedelta(hours=15, minutes=25))]
    else:
        eq_filtered = df_eq[df_eq['date'] <= curr_date + pd.Timedelta(hours=15, minutes=25)]

    # Compute required values
    num_list.append(len(eq_filtered))
    sum_list.append(eq_filtered['magnitudo'].sum() if not eq_filtered.empty else np.nan)
    max_list.append(eq_filtered['magnitudo'].max() if not eq_filtered.empty else np.nan)
    avg_list.append(eq_filtered['magnitudo'].mean() if not eq_filtered.empty else np.nan)

    # Compute distances from London
    if not eq_filtered.empty:
        distances = eq_filtered.apply(lambda row: geodesic((row['latitude'], row['longitude']), tokyo_coords).km, axis=1)
        min_dist_list.append(distances.min())
        max_dist_list.append(distances.max())
    else:
        min_dist_list.append(np.nan)
        max_dist_list.append(np.nan)

# Create the combined df
df = df_n225.copy()
df['num'] = num_list
df['sum'] = sum_list
df['max'] = max_list
df['avg'] = avg_list
df['min_dist'] = min_dist_list
df['max_dist'] = max_dist_list

# Display result
print(df.head())


  Ticker                      date      change  num  sum  max  avg  \
0  ^N225 2008-08-01 00:00:00+09:00 -181.980469    0  NaN  NaN  NaN   
1  ^N225 2008-08-04 00:00:00+09:00 -150.100586    0  NaN  NaN  NaN   
2  ^N225 2008-08-05 00:00:00+09:00  -42.349609    1  6.3  6.3  6.3   
3  ^N225 2008-08-06 00:00:00+09:00  195.459961    1  6.0  6.0  6.0   
4  ^N225 2008-08-07 00:00:00+09:00 -133.000000    0  NaN  NaN  NaN   

      min_dist     max_dist  
0          NaN          NaN  
1          NaN          NaN  
2  4818.209478  4818.209478  
3  3159.157480  3159.157480  
4          NaN          NaN  


In [4]:
#Final preparations of the dataframe

#Changing the missing values to 0. Have to see if not doing this is preferable
df[['sum', 'max', 'avg', 'min_dist', 'max_dist']] = df[['sum', 'max', 'avg', 'min_dist', 'max_dist']].fillna(0)

#In this model I used 1 divided by minimum distance of the earthquakes in each time period as one of the factors, assuming the further the earthquake, the lesser its effect
df['rev_dist'] = df['min_dist'].apply(lambda x: 0 if x == 0 else 1 / x)
#Creating a csv file to be able to looking at it by Excel, as it might be faster/easier to go through
df.to_csv('data/test/testn225.csv')

,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,rev_dist
0,^N225,2008-08-01 00:00:00+09:00,-181.980469,0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,^N225,2008-08-04 00:00:00+09:00,-150.100586,0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,^N225,2008-08-05 00:00:00+09:00,-42.349609,1,6.3,6.3,6.3,4818.209478,4818.209478,0.000208
3,^N225,2008-08-06 00:00:00+09:00,195.459961,1,6.0,6.0,6.0,3159.157480,3159.157480,0.000317
4,^N225,2008-08-07 00:00:00+09:00,-133.000000,0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [5]:
#This part does the regreesion modeling
import statsmodels.api as sm

# Define independent variables (num, max, rev_dist) and dependent variable (change)
X = df[['num', 'max', 'rev_dist']]
y = df['change']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 change   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.960
Date:                Tue, 04 Feb 2025   Prob (F-statistic):              0.120
Time:                        15:34:57   Log-Likelihood:                -2276.7
No. Observations:                 344   AIC:                             4561.
Df Residuals:                     340   BIC:                             4577.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -26.9750     13.027     -2.071      0.0